**Note**: There are multiple ways to solve these problems in SQL. Your solution may be quite different from mine and still be correct.

**1**. Connect to the SQLite3 database at `data/faculty.db` in the `notebooks` folder using the `sqlite` package or `ipython-sql` magic functions. Inspect the `sql` creation statement for each tables so you know their structure.

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///../notebooks/data/faculty.db

In [ ]:
%%sql

SELECT sql FROM sqlite_master WHERE type='table';

2. Find the youngest and oldest faculty member(s) of each gender.

In [ ]:
%%sql

SELECT min(age), max(age) FROM person

In [ ]:
%%sql

SELECT first, last, age, gender
FROM person 
INNER JOIN gender 
    ON person.gender_id = gender.gender_id
WHERE age IN (SELECT min(age) FROM person) AND gender = 'Male'
UNION
SELECT first, last, age, gender
FROM person 
INNER JOIN gender 
    ON person.gender_id = gender.gender_id
WHERE age IN (SELECT min(age) FROM person) AND gender = 'Female'
UNION
SELECT first, last, age, gender
FROM person 
INNER JOIN gender 
    ON person.gender_id = gender.gender_id
WHERE age IN (SELECT max(age) FROM person) AND gender = 'Male'
UNION
SELECT first, last, age, gender
FROM person 
INNER JOIN gender 
    ON person.gender_id = gender.gender_id
WHERE age IN (SELECT max(age) FROM person) AND gender = 'Female'
LIMIT 10

3. Find the median age of the faculty members who know Python.

As SQLite3 does not provide a median function, you can create a User Defined Function (UDF) to do this. See [documentation](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection.create_function).

In [ ]:
import statistics

In [ ]:
class Median:
    def __init__(self):
        self.acc = []

    def step(self, value):
        self.acc.append(value)

    def finalize(self):
        return statistics.median(self.acc)

In [ ]:
import sqlite3
con = sqlite3.connect('../notebooks/data/faculty.db')
con.create_aggregate("Median", 1, Median)

In [ ]:
cr = con.cursor()
cr.execute('SELECT median(age) FROM person')
cr.fetchall()

4. Arrange countries by the average age of faculty in descending order. Countries are only included in the table if there are at least 3 faculty members from that country.

In [ ]:
%%sql

SELECT country, count(country), avg(age)
FROM person
INNER JOIN country
    ON person.country_id = country.country_id
GROUP BY country
HAVING count(*) > 3
ORDER BY age DESC
LIMIT 3

5. Which country has the highest average body mass index (BMII) among the faculty? Recall that BMI is weight (kg) / (height (m))^2.

In [ ]:
%%sql

SELECT country, avg(weight / (height*height)) as avg_bmi
FROM person
INNER JOIN country
    ON person.country_id = country.country_id
GROUP BY country
ORDER BY avg_bmi DESC
LIMIT 3

6. Do obese faculty (BMI > 30) know more languages on average than non-obese faculty?

In [ ]:
%%sql

SELECT is_obese, avg(language)
FROM (
    SELECT 
        weight / (height*height) > 30 AS is_obese, 
        count(language_name) AS language
    FROM person
    INNER JOIN person_language
        ON person.person_id = person_language.person_id
    INNER JOIN language
        ON person_language.language_id = language.language_id
    GROUP BY person.person_id
)
GROUP BY is_obese